In [ ]:
import requests
# from bs4 import BeautifulSoup
import socket    
from tenacity import retry, stop_after_delay, wait_fixed
import time
from n26_api import Api
import sys
import yaml
print(sys.argv[1:])
%autoreload 2

# N26

In [ ]:
GET = "get"
POST = "post"

EXPIRATION_TIME_KEY = "expiration_time"
ACCESS_TOKEN_KEY = "access_token"
REFRESH_TOKEN_KEY = "refresh_token"

GRANT_TYPE_PASSWORD = "password"
GRANT_TYPE_REFRESH_TOKEN = "refresh_token"
BASE_URL_DE = "https://api.tech26.de"
BASE_URL_GLOBAL = "https://api.tech26.global"
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
# BASIC_AUTH_HEADERS = {"Authorization": "Basic bmF0aXZld2ViOg==", "Content-Type": "application/json", "User-Agent": USER_AGENT}
# BASIC_AUTH_HEADERS = {"Authorization": "Basic bmF0aXZld2ViOg==", "Content-Type": "application/x-www-form-urlencoded", "User-Agent": USER_AGENT}
BASIC_AUTH_HEADERS = {"Authorization": "Basic bmF0aXZld2ViOg=="}

In [ ]:
response = requests.head(f"{BASE_URL_GLOBAL}/oauth2/token")
response.headers["x-request-id"]

In [ ]:
import os.path
import time
import csv
FIELD_NAMES = ["refresh_token", "expires_in", "time"]

In [ ]:
file_exists = os.path.isfile("refresh_token.csv")
print(file_exists)
with open("refresh_token.csv", "a") as f:
    # csv_writer = csv.writer(f)
    csv_writer = csv.DictWriter(f, fieldnames=FIELD_NAMES, delimiter=",")
    if not file_exists:
        print("in")
        csv_writer.writeheader()
    line_to_write = {
        "refresh_token": 124,
        "expires_in": 5,
        "time": time.time(),
    }
    csv_writer.writerow(line_to_write)

In [ ]:
FIELD_NAMES = ["refresh_token", "expires_in", "time"]
file_exists = os.path.isfile("refresh_token.csv")
with open("./refresh_token.csv", "a") as f:
    csv_writer = csv.DictWriter(f, fieldnames=FIELD_NAMES, delimiter=",")
    if not file_exists:
        csv_writer.writeheader()
    line_to_write = {
        "refresh_token": 3,
        "expires_in": 3,
        "time":4,
    }
    csv_writer.writerow(line_to_write)

In [ ]:
history_of_refresh_token=1
print(f"history_of_refresh_token: {history_of_refresh_token}\n")

In [ ]:
file_exists = os.path.isfile("./refresh_token.csv")
print(file_exists)

In [ ]:
def first_try():
    try:
        with open("./refresh_token.csv", "r") as f:
            csv_reader = csv.DictReader(f)
            if REFRESH_TOKEN_KEY in csv_reader.fieldnames:
                history_of_refresh_token = [line[REFRESH_TOKEN_KEY] for line in csv_reader]
                # self.token_data = {REFRESH_TOKEN_KEY: history_of_refresh_token[-1]}
                print(f"history_of_refresh_token: {history_of_refresh_token}\n")
                return True
            else:
                print("nop")
                # raise AssertionError("Cant refresh token since no existing token data was found in file.Please initiate a new authentication instead.")
    # except (IOError, OSError, FileNotFoundError) as error:
    except OSError as error:
        # print("holi")
        raise OSError("holi")
            # "Cant refresh token since no file to read existen token data was found. Please initiate a new authentication instead.",error)

In [ ]:
first_try()

In [ ]:
def sec_try():
    try:
        first_try()
    except OSError:
        print("q weaa")


    

In [ ]:
sec_try()

In [ ]:
first_try().strerror == 'No such file or directory'

In [ ]:
first_try().strerror

In [ ]:
dir(first_try())

In [ ]:
sec_try()

In [ ]:
with open("./refresh_token.csv", "r") as f:
    csv_reader = csv.DictReader(f)
    if "refresh_token" in csv_reader.fieldnames:
        history_of_refresh_token = [line["refresh_token"] for line in csv_reader]
        print(f"history_of_refresh_token: {history_of_refresh_token}\n")
    else:
        raise ValueError("No refresh token was found in the columns")
    except Exception:
        print('not found')

In [ ]:
response = requests.post(f"{BASE_URL_DE}/api/mfa/challenge")
response

In [ ]:
response.json()

In [ ]:
response.status_code == 401

In [ ]:
response.json()

In [ ]:
raise ValueError(
    "Unexpected response for initial auth request: {}".format(response.text)
)

In [ ]:
user_1 = Api()

In [ ]:
user_1.authenticate()

In [ ]:
with open("./credentials.yaml", "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


In [ ]:
VALUES_TOKEN = config["values_token"]

# get mfa token

In [ ]:
f"{BASE_URL_DE}/oauth2/token"

In [ ]:
s = requests.session() # a sessions is going to hold a lot of data
BASIC_AUTH_HEADERS["device-token"] = s.head(BASE_URL_GLOBAL+"/oauth2/token").headers["x-request-id"]
response = s.post(BASE_URL_DE+"/oauth2/token", data=VALUES_TOKEN, headers=BASIC_AUTH_HEADERS)
if response.status_code != 403:
    raise ValueError("Unexpected response for initial auth request: {}".format(response.text))
mfa_token = response.json()["mfaToken"]
mfa_data = {
    "mfaToken": mfa_token,
    "challengeType": "oob",
}
response.text

# get push notifications in paired device

In [ ]:
response = s.post(BASE_URL_DE+"/api/mfa/challenge",json=mfa_data,headers={**BASIC_AUTH_HEADERS,"User-Agent": USER_AGENT,"Content-Type": "application/json"})
response.text

# Get access and refresh token. Accept in paired device

In [ ]:
mfa_token

In [ ]:
@retry(wait=wait_fixed(5), stop=stop_after_delay(60))
def finish_auth():
    mfa_response_data = {
        "mfaToken": mfa_token,
        "grant_type": "mfa_oob",
        
    }
    response = s.post(BASE_URL_DE+"/oauth2/token",data=mfa_response_data,headers=BASIC_AUTH_HEADERS)
    response.raise_for_status()
    tokens = response.json()
    return tokens

In [ ]:
def is_authenticated(self) -> bool:
    return self._validate_token(self.token_data)


In [ ]:
import time

In [ ]:
h = {
    "access_token":1,
    "token_type": 2,
    "refresh_token": 3,
    "expires_in": 4,
    "scope": 5,
    "host_url": 6,
}

In [ ]:
from pathlib import Path


In [ ]:
from pathlib import Path


In [ ]:
def _refresh_token(refresh_token: str):
    values_token = {
        'grant_type': GRANT_TYPE_REFRESH_TOKEN,
        'refresh_token': refresh_token,
    }
    response = requests.post(BASE_URL_GLOBAL + '/oauth2/token', data=values_token, headers=BASIC_AUTH_HEADERS)
    response.raise_for_status()
    return response.json()


In [ ]:
token_data = finish_auth()
token_data["expiration_time"] = time.time() + token_data["expires_in"]
refresh_token = token_data["refresh_token"]
# _refresh_token(refresh_token)
